Todo:
- vyplnit nulls pro narození nejpřesnějším známým datem

In [434]:
import os
import polars as pl

In [435]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [436]:
df = pl.read_ndjson(os.path.join('data_raw/poslanectvo','*.ndjson'), schema_overrides={'navrhovatel':pl.String,'kandidatka':pl.String})

In [437]:
df.sample(100)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni,clenstvi
str,str,str,str,str,str,str,str,str,str,str,list[struct[4]]
"""437""","""5""","""437_5.html""","""Ing. Martin Říman""","""Moravskoslezský""","""m""","""3. 6. 2006""","""3. 6. 2010""","""ODS""",null,"""1961""","[{""člen"",""Petiční výbor"",""5. 11. 2009"",""3. 6. 2010""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""8. 6. 2006"",""3. 6. 2010""}, … {null,""Vláda České republiky"",""9. 1. 2007"",""8. 5. 2009""}]"
"""297""","""2""","""297_2.html""","""RNDr. Vilém Holáň""","""Severomoravský""","""m""","""1. 6. 1996""","""19. 6. 1998""","""KDU-ČSL""",null,"""1938""","[{""člen"",""Zahraniční výbor"",""2. 7. 1996"",""19. 6. 1998""}, {""člen"",""Organizační výbor"",""2. 7. 1996"",""19. 6. 1998""}, … {""člen"",""Vláda České republiky"",""1. 1. 1993"",""3. 7. 1996""}]"
"""162""","""1""","""162_1.html""","""Doc. RSDr. Jaroslav Štrait, CSc.""","""Východočeský""","""m""","""6. 6. 1992""","""6. 6. 1996""",null,null,"""1942""","[{""člen"",""Výbor pro sociální politiku a zdravotnictví"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Levého bloku"",""6. 6. 1992"",""18. 1. 1994""}, {""člen"",""Poslanecký klub Komunistické strany Čech a Moravy"",""19. 1. 1994"",""6. 6. 1996""}]"
"""5888""","""5""","""5888_5.html""","""JUDr. Klára Slámová""","""Hlavní""","""f""","""30. 4. 2010""","""3. 6. 2010""","""ODS""",null,"""1977""","[{""člen"",""Nezařazení"",""30. 4. 2010"",""3. 6. 2010""}]"
"""250""","""3""","""250_3.html""","""Ing. Miloslav Vlček""","""Jihomoravský""","""m""","""20. 6. 1998""","""20. 6. 2002""","""ČSSD""",null,"""1961""","[{""člen"",""Rozpočtový výbor"",""22. 7. 1998"",""20. 6. 2002""}, {""člen"",""Podvýbor pro kontrolu"",""23. 9. 1998"",""20. 6. 2002""}, … {""člen"",""Dozorčí rada České konsolidační agentury"",""10. 7. 2001"",""10. 7. 2006""}]"
"""6203""","""7""","""6203_7.html""","""Mgr. et Mgr. Věra Jourová""","""Vysočina""","""f""","""26. 10. 2013""","""21. 10. 2014""","""ANO2011""",null,"""1964""","[{""člen"",""Ústavně právní výbor"",""4. 12. 2013"",""29. 1. 2014""}, {""člen"",""Poslanecký klub ANO 2011"",""30. 10. 2013"",""21. 10. 2014""}, {null,""Vláda České republiky"",""29. 1. 2014"",""3. 10. 2014""}]"
"""420""","""3""","""420_3.html""","""Ladislav Šustr""","""Jihomoravský""","""m""","""20. 6. 1998""","""20. 6. 2002""","""KDU-ČSL""",null,"""1944""","[{""člen"",""Rozpočtový výbor"",""22. 7. 1998"",""20. 6. 2002""}, {""člen"",""Podvýbor pro tělovýchovu"",""16. 9. 1998"",""20. 6. 2002""}, … {""člen"",""Poslanecký klub Křesťanské a demokratické unie - Československé strany lidové"",""25. 6. 1998"",""20. 6. 2002""}]"
"""64""","""2""","""64_2.html""","""Ing. Jan Kasal""","""Jihomoravský""","""m""","""1. 6. 1996""","""19. 6. 1998""","""KDU-ČSL""",null,"""1951""","[{""člen"",""Organizační výbor"",""2. 7. 1996"",""19. 6. 1998""}, {""místopředsed"",""Organizační výbor"",""2. 7. 1996"",""19. 6. 1998""}, … {""člen"",""Poslanecký klub Křesťanské a demokratické unie - Československá strany lidové"",""6. 6. 1996"",""19. 6. 1998""}]"
"""6142""","""7""","""6142_7.html""","""Mgr. Daniel Herman""","""Hlavní""","""m""","""26. 10. 2013""","""26. 10. 2017""","""KDU-ČSL""",null,"""1963""","[{""člen"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""29. 1. 2014""}, {""člen"",""Volební výbor"",""4. 12. 2013"",""29. 1. 2014""}, … {null,""Vláda České republiky"",""29. 1. 2014"",""13. 12. 2017""}]"


In [438]:
obdobi = {
    '9':'2021',
    '8':'2017',
    '7':'2013',
    '6':'2010',
    '5':'2006',
    '4':'2002',
    '3':'1998',
    '2':'1996',
    '1':'1992'
}

In [439]:
def najdi_tituly(jmeno):
    jmeno = jmeno.replace('ml.','mladší').replace('st.','starší')
    tituly = []
    za = None
    if ',' in jmeno:
        tituly_za = ' '.join(jmeno.split(',')[1:])
        za = [t.strip() for t in tituly_za.strip().split(' ') if t != '']
        jmeno = jmeno.split(',')[0]
    pred = [t for t in jmeno.split(' ') if '.' in t]
    jmeno = jmeno.split('.')[-1].strip()
    if za:
        tituly = pred + za
    else:
        tituly = pred
    return {'jmeno_ciste':jmeno,'tituly':tituly}

In [440]:
najdi_tituly("PhDr. Walter Bartoš, Ph.D., MBA")

{'jmeno_ciste': 'Walter Bartoš', 'tituly': ['PhDr.', 'Ph.D.', 'MBA']}

In [441]:
len(df)

1606

In [442]:
df.filter(pl.col('jmeno').str.contains('Benda'))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni,clenstvi
str,str,str,str,str,str,str,str,str,str,str,list[struct[4]]
"""4""","""1""","""4_1.html""","""Marek Benda""","""Středočeský""","""m""","""6. 6. 1992""","""6. 6. 1996""",null,null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""1. 4. 1996"",""6. 6. 1996""}]"
"""4""","""2""","""4_2.html""","""Marek Benda""","""Severomoravský""","""m""","""1. 6. 1996""","""19. 6. 1998""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""27. 6. 1996"",""19. 6. 1998""}, {""člen"",""Ústavně právní výbor"",""2. 7. 1996"",""19. 6. 1998""}, … {""místopředsed"",""Poslanecký klub Občanské demokratické strany"",""16. 7. 1996"",""9. 6. 1997""}]"
"""4""","""3""","""4_3.html""","""Marek Benda""","""Severomoravský""","""m""","""20. 6. 1998""","""20. 6. 2002""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""15. 7. 1998"",""20. 6. 2002""}, {""člen"",""Ústavně právní výbor"",""22. 7. 1998"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Tunisko"",""1. 4. 1999"",""20. 6. 2002""}]"
"""4""","""4""","""4_4.html""","""Marek Benda""","""Hlavní""","""m""","""20. 7. 2004""","""15. 6. 2006""","""ODS""",null,"""1968""","[{""člen"",""Ústavně právní výbor"",""24. 9. 2004"",""15. 6. 2006""}, {""člen"",""Podvýbor pro kriminalitu mládeže"",""31. 3. 2005"",""15. 6. 2006""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""20. 7. 2004"",""15. 6. 2006""}]"
"""4""","""5""","""4_5.html""","""Marek Benda""","""Hlavní""","""m""","""3. 6. 2006""","""3. 6. 2010""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""29. 6. 2006"",""3. 6. 2010""}, {""místopředsed"",""Mandátový a imunitní výbor"",""29. 6. 2006"",""15. 9. 2006""}, … {""člen"",""Skupina ČR - Maďarsko"",""7. 5. 2008"",""3. 6. 2010""}]"
"""4""","""6""","""4_6.html""","""Marek Benda""","""Hlavní""","""m""","""29. 5. 2010""","""28. 8. 2013""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""24. 6. 2010"",""28. 8. 2013""}, {""člen"",""Ústavně právní výbor"",""7. 7. 2010"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - Arménie"",""9. 5. 2012"",""28. 8. 2013""}]"
"""4""","""7""","""4_7.html""","""Marek Benda""","""Hlavní""","""m""","""26. 10. 2013""","""26. 10. 2017""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""12. 12. 2013""}, {""předsed"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""12. 12. 2013""}, … {""člen"",""Skupina ČR - Rumunsko, Moldavská republika"",""26. 11. 2014"",""26. 10. 2017""}]"
"""4""","""8""","""4_8.html""","""Marek Benda""","""Hlavní""","""m""","""21. 10. 2017""","""20. 10. 2021""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""22. 11. 2017"",""20. 10. 2021""}, {""člen"",""Ústavně právní výbor"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Izrael"",""10. 4. 2018"",""20. 10. 2021""}]"
"""4""","""9""","""4_9.html""","""Marek Benda""","""Hlavní""","""m""","""9. 10. 2021""",null,"""SPOLU""","""ODS""","""10. 11. 1968""","[{""člen"",""Mandátový a imunitní výbor"",""10. 11. 2021"",null}, {""člen"",""Zahraniční výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Albánská republika"",""15. 6. 2022"",null}]"


In [443]:
df = df.with_columns(pl.col("kraj").replace("Hlavní","Praha")).with_columns(
    pl.col(["obdobi_od","obdobi_do"]).str.to_date()
).rename(
    {'narozeni':'narozeni_retezec'}
).with_columns(
    pl.col('narozeni_retezec').str.extract(r'\d{4}',0).cast(int).alias('narozeni_rok')
).with_columns(
    pl.col('narozeni_retezec').str.to_date(strict=False).alias('narozeni_datum')
).with_columns(
    pl.col('obdobi').replace(obdobi).cast(pl.Int32)
).sort(
    by=['obdobi','id']
).with_columns(
    pl.col('jmeno').map_elements(najdi_tituly,return_dtype=pl.Struct).alias('jmeno+tituly')
).unnest(
    "jmeno+tituly"
)

In [444]:
len(df)

1606

In [445]:
len(df.unique(subset=['id','jmeno']))

854

In [446]:
df.group_by("kandidatka").len()

kandidatka,len
str,u32
"""K""",34
"""Piráti""",23
"""ODS Hlavní menu Přehled obsahu Společná česko-slovenskádigitální parlamentní knihovna Digitální knihovnaČeské sněmy Informace Rejstříky Zajímav…",1
"""ODA""",2
"""SZ""",6
"""PirSTAN""",39
null,37
"""SPOLU""",72
"""VV""",24


In [447]:
df.group_by("navrhovatel").len()

navrhovatel,len
str,u32
"""KDU-ČSL""",46
"""STAN""",34
"""US""",10
"""ODS""",35
"""TOP09""",14
"""Piráti""",4
null,1463


In [448]:
df.group_by('obdobi').agg(pl.col('id').unique().len()).sort(by='obdobi')

obdobi,id
i32,u32
1992,37
1996,100
1998,168
2002,234
2006,220
2010,213
2013,208
2017,216
2021,210


In [478]:
df.group_by('obdobi').agg(pl.col('jmeno').unique().len()).sort(by='obdobi')

obdobi,jmeno
i32,u32
1992,37
1996,100
1998,168
2002,234
2006,220
2010,213
2013,208
2017,216
2021,210


In [449]:
najdi_tituly("RSDr. Ing. Svatomír Recman")

{'jmeno_ciste': 'Svatomír Recman', 'tituly': ['RSDr.', 'Ing.']}

In [450]:
df.group_by(['jmeno','obdobi']).len().sort(by='len',descending=True)

jmeno,obdobi,len
str,i32,u32
"""Ing. Radim Fiala""",2010,1
"""Ing. Petr Zgarba""",2002,1
"""Ing. David Kafka""",2006,1
"""RNDr. Michal Lobkowicz""",1996,1
"""Ing. Věra Kovářová""",2021,1
"""Ing. Jaromír Kohlíček, CSc.""",2002,1
"""Mgr. Jana Pastuchová""",2013,1
"""Ing. Ladislav Býček""",2002,1
"""RNDr. Vilém Holáň""",1996,1


In [480]:
najdi_tituly("prof. Ing. Karel Rais, CSc., MBA")

{'jmeno_ciste': 'Karel Rais', 'tituly': ['prof.', 'Ing.', 'CSc.', 'MBA']}

In [451]:
df.filter(pl.col('jmeno_ciste') == '')

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]


In [452]:
df.group_by(['jmeno_ciste','obdobi']).len().sort(by='len',descending=True)

jmeno_ciste,obdobi,len
str,i32,u32
"""Bohuslav Sobotka""",2002,1
"""Adam Kalous""",2017,1
"""Jana Bačíková""",2021,1
"""Vladimír Čada""",2002,1
"""Lubomír Volný""",2017,1
"""Milan Šťovíček""",2010,1
"""Jan Zahradník""",2013,1
"""Hana Orgoníková""",1998,1
"""Miloslav Rozner""",2017,1


In [453]:
df

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""100""",1992,"""100_1.html""","""JUDr. Hana Kordová Marvanová""","""Praha""","""f""",1992-06-06,1996-06-06,null,null,"""1962""","[{""člen"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""místopředsed"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1962,null,"""Hana Kordová Marvanová""","[""JUDr.""]"
"""105""",1992,"""105_1.html""","""JUDr. Dalibor Matulka""","""Jihomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1953""","[{""člen"",""Mandátový a imunitní výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Komunistické strany Čech a Moravy"",""19. 1. 1994"",""6. 6. 1996""}]",1953,null,"""Dalibor Matulka""","[""JUDr.""]"
"""110""",1992,"""110_1.html""","""Mgr. Václav Nájemník""","""Východočeský""","""m""",1992-06-06,1996-06-06,null,null,"""1944""","[{""člen"",""Výbor pro veřejnou správu, regionální rozvoj a životní prostředí"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1944,null,"""Václav Nájemník""","[""Mgr.""]"
"""111""",1992,"""111_1.html""","""RNDr. Petr Nečas""","""Severomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1964""","[{""člen"",""Zahraniční výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Stálá komise pro reformu veřejné správy"",""29. 9. 1994"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1964,null,"""Petr Nečas""","[""RNDr.""]"
"""12""",1992,"""12_1.html""","""Ing. František Brožík""","""Západočeský""","""m""",1992-06-06,1996-06-06,null,null,"""1955""","[{""člen"",""Hospodářský výbor"",""6. 6. 1992"",""6. 6. 1996""}, {null,""Hospodářský výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""6. 6. 1992"",""6. 6. 1996""}]",1955,null,"""František Brožík""","[""Ing.""]"
"""122""",1992,"""122_1.html""","""Ing. Hana Orgoníková""","""Východočeský""","""f""",1992-06-06,1996-06-06,null,null,"""1946""","[{""člen"",""Výbor petiční, pro lidská práva a národnosti"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Stálá komise pro práci Kanceláře Poslanecké sněmovny"",""27. 1. 1993"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""12. 10. 1994"",""6. 6. 1996""}]",1946,null,"""Hana Orgoníková""","[""Ing.""]"
"""124""",1992,"""124_1.html""","""Ing. Jaroslav Palas""","""Severomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1952""","[{""člen"",""Zemědělský výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Levého bloku"",""6. 6. 1992"",""27. 9. 1995""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""1. 11. 1995"",""6. 6. 1996""}]",1952,null,"""Jaroslav Palas""","[""Ing.""]"
"""127""",1992,"""127_1.html""","""RNDr. Jiří Payne""","""Praha""","""m""",1992-06-06,1996-06-06,null,null,"""1956""","[{""člen"",""Zahraniční výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""předsed"",""Zahraniční výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1956,null,"""Jiří Payne""","[""RNDr.""]"
"""136""",1992,"""136_1.html""","""RSDr. Ing. Svatomír Recman""","""Severomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1952""","[{""člen"",""Rozpočtový výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Stálá komise pro práci Kanceláře Poslanecké sněmovny"",""27. 1. 1993"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Komunistické strany Čech a Moravy"",""15. 2. 1994"",""6. 6. 1996""}]",1952,null,"""Svatomír Recman""","[""RSDr."", ""Ing.""]"


In [454]:
df.write_parquet(os.path.join("data","mluvci.parquet"))